# 1장 케라스 시작

In [ ]:
import keras

print(keras.backend.backend())

In [ ]:
import numpy as np

x = np.array([0, 1, 2, 3, 4])
y = x * 2 + 1

In [ ]:
import keras

model = keras.models.Sequential()
model.add(keras.layers.Dense(1, input_shape=(1, )))
model.compile(optimizer='SGD', loss='mse')

model.fit(x[:2], y[:2], epochs=1000, verbose=0)

print('Targets:', y[2:])
print('Predictions:', model.predict(x[2:]).flatten())

# 2장 ANN

In [ ]:
# 분산 방식 모델링, 함수형 구현
from keras import layers, models

x = layers.Input(shape=(None, ))
h = layers.Activation('relu')(layers.Dense(None)(x))
y = layers.Activation('softmax')(layers.Dense(None)(h))

model = models.Model(x, y)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 연쇄 방식 모델링, 함수형 구현
model = models.Sequential() # 모델 구조 정의 전 Sequential로 초기화
model.add(layers.Dense(None, activation='relu', input_shape=(None, )))
model.add(layers.Dense(None, activation='softmax'))

In [ ]:
# 분산 방식 모델링, 객체지향형 구현
class ANN(models.Model):
    def __init__(self, input_num, hidden_num, output_num, **kwargs):
        hidden = layers.Dense(hidden_num)
        output = layers.Dense(output_num)
        relu = layers.Activation('relu')
        softmax = layers.Activation('softmax')
        super().__init__(**kwargs)

model = ANN(input_num, hidden_num, output_num)

In [ ]:
# 연쇄 방식 모델링, 객체지향형 구현
class ANN(models.Sequential):
    def __init__(self, input_num, hidden_num, output_num, **kwargs):
        super().__init__(**kwargs)
        self.add(layers.Dense(hidden_num, activation='relu', input_shape=(input_num, )))
        self.add(layers.Dense(output_num, activation='softmax'))
        self.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import numpy as np
# from keras import datasets  #mnist
from keras.datasets import mnist    # 이렇게 불러와야 실행됨
from keras.utils import np_utils # to_categorical

def Data_func():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)

    L, W, H = X_train.shape
    X_train = X_train.reshape(-1, W*H)
    X_test = X_test.reshape(-1, W*H)

    X_train = X_train/255.0
    X_test = X_test/255.0

    return (X_train, y_train), (X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc=0)

def plot_acc(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc=0)

In [ ]:
input_num = 784
hidden_num = 100
number_of_class = 10
output_num = number_of_class
model = ANN(input_num, hidden_num, output_num)
(X_train, y_train), (X_test, y_test) = Data_func()

history = model.fit(X_train, y_train, epochs=5, batch_size=100, validation_split=0.2)

performance_test = model.evaluate(X_test, y_test, batch_size=100)
print('Test loss and Accuracy -> {: .2f}, {: .2f}'.format(*performance_test))

In [ ]:
plot_loss(history)
plt.show()
plot_acc(history)
plt.show()

In [ ]:
# 시계열
from keras import layers, models

class ANN(models.Model):
    def __init__(self, Nin, Nh, Nout):
        hidden = layers.Dense(Nh)
        output = layers.Dense(Nout)
        relu = layers.Activation('relu')

        x = layers.Input(shape=(Nin, ))
        h = relu(hidden(x))
        y = output(h)

        super.__init__(x, y)
        self.compile(loss='mse', optimizer='sgd')

# 보스턴 집값 데이터
from keras.datasets import boston_housing
from sklearn import preprocessing

def Data_func():
    (X_train, y_train), (X_test, y_test) = boston_housing.load_data()
    scaler = preprocessing.MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return (X_train, y_train), (X_test, y_test)

# 시각화
from original.keraspp.skeras import plot_loss
import matplotlib.pyplot as plt

def main():
    Nin = 13
    Nh = 5
    Nout = 1

    model = ANN(Nin, Nh, Nout)
    (X_train, y_train), (X_test, y_test) = Data_func()
    history = model.fit(X_train, y_train, epochs=100, batch_size=100, validation_split=0.2, verbose=2)

    performance_test = model.evaluate(X_test, y_test, batch_size=100)
    print('\nTest Loss -> {: .2f}'.format(performance_test))

    plot_loss()
    plt.show()

# 3장 DNN

In [ ]:
from keras import layers, models

Nin = 784
Nh_l = [100, 50]
number_of_class = 10
Nout = number_of_class

class DNN(models.Sequential):
    def __init__(self, Nin, Nh_1, Nout):
        super().__init__()

        self.add(layers.Dense(Nh_1[0], activation='relu', input_shape=(Nin, ), name='Hidden-1'))
        self.add(layers.Dropout(0.2))
        self.add(layers.Dense(Nh_1[1], activation='relu', name='Hidden-2'))
        self.add(layers.Dropout(0.2))
        self.add(layers.Dense(Nout, activation='softmax'))

        self.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils # to_categorical

def Data_func():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)

    L, W, H = X_train.shape
    X_train = X_train.reshape(-1, W*H)
    X_test = X_test.reshape(-1, W*H)

    X_train = X_train/255.0
    X_test = X_test/255.0

    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = Data_func()

model = DNN(Nin, Nh_l, Nout)
history = model.fit(X_train, y_train, epochs=10, batch_size=100, validation_split=0.2)
performance_test = model.evaluate(X_test, y_test, batch_size=100)
print('Test Loss and Accuracy ->', performance_test)

In [ ]:
# 컬러 이미지
import numpy as np
from keras.datasets import cifar10
from keras.utils import np_utils

def Data_func():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)

    L, W, H, C = X_train.shape
    X_train = X_train.reshape(-1, W*H*C)
    X_test = X_test.reshape(-1, W*H*C)

    X_train = X_train/255.0
    X_test = X_test/255.0

    return (X_train, y_train), (X_test, y_test)


# 모델링
from keras import layers, models

class DNN(models.Sequential):
    def __init__(self, Nin, Nh_l, Pd_l, Nout):
        super().__init__()

        self.add(layers.Dense(Nh_l[0], activation='relu', input_shape=(Nin, ), name='Hiiden-1'))
        self.add(layers.Dropout(Pd_l[0]))
        self.add(layers.Dense(Nh_l[1], activation='relu', input_shape=(Nin, ), name='Hiiden-2'))
        self.add(layers.Dropout(Pd_l[1]))
        self.add(layers.Dense(Nout, activation='softmax'))
        self.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# 학습 및 시각화
from original.keraspp.skeras import plot_loss, plot_acc
import matplotlib.pyplot as plt

Nh_l = [100, 50]
Pd_l = [0.0, 0.0]
number_of_class = 10
Nout = number_of_class

(X_train, y_train), (X_test, y_test) = Data_func()
model = DNN(X_train.shape[1], Nh_l, Pd_l, Nout)
history = model.fit(X_train, y_train, epochs=10, batch_size=100, validation_split=0.2)
performance_test = model.evaluate(X_test, y_test, batch_size=100)
print('Test Loss and Accuracy ->', performance_test)

plot_acc(history)
plt.show(block=False)
plt.pause(2)
plt.close()
plot_loss(history)
plt.show(block=False)
plt.pause(2)
plt.close()

# 4장 CNN

In [ ]:
import keras
from keras import models, layers
import keras.backend
import keras.optimizers

class CNN(models.Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        # convolutionL 특징점 찾기
        self.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
        self.add(layers.Conv2D(64, (3,3), activation='relu'))
        self.add(layers.MaxPooling2D(pool_size=(2,2)))  # pool_size: 1개의 값으로 변화할 크기
        self.add(layers.Dropout(0.25))
        self.add(layers.Flatten())

        # dense: 실질적 분류 작업
        self.add(layers.Dense(128, activation='relu'))
        self.add(layers.Dropout(0.5))
        self.add(layers.Dense(num_classes, activation='softmax'))

        self.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['acc'])

from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 채널의 위치는 image_data_format에 따름
print(keras.backend.backend_config.image_data_format())

# 1 = 채널의 크기
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_test = X_test.reshape(*X_test.shape, 1)
input_shape = (X_train.shape[1], X_train.shape[2], 1)

## 채널의 위치가 앞 단에 존재할 경우
# X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])
# X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2])
# input_shape = (1, X_train.shape[1], X_train.shape[2])

In [ ]:
import keras
from keras import models, layers
import keras.backend
import keras.optimizers

class CNN(models.Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        # convolutionL 특징점 찾기
        self.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
        self.add(layers.Conv2D(64, (3,3), activation='relu'))
        self.add(layers.MaxPooling2D(pool_size=(2,2)))  # pool_size: 1개의 값으로 변화할 크기
        self.add(layers.Dropout(0.25))
        self.add(layers.Flatten())

        # dense: 실질적 분류 작업
        self.add(layers.Dense(128, activation='relu'))
        self.add(layers.Dropout(0.5))
        self.add(layers.Dense(num_classes, activation='softmax'))

        self.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['acc'])

from keras.datasets import mnist
class DATA():
    def __init__(self):
        num_classes = 10

        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        
        # 채널의 위치는 image_data_format에 따름
        print(keras.backend.backend_config.image_data_format())

        # 1 = 채널의 크기
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
        # X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
        X_test = X_test.reshape(*X_test.shape, 1)
        input_shape = (X_train.shape[1], X_train.shape[2], 1)

        ## 채널의 위치가 앞 단에 존재할 경우
        # X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])
        # X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2])
        # input_shape = (1, X_train.shape[1], X_train.shape[2])

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255

        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)
        
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.X_train, self.y_train = X_train, y_train
        self.X_test, self.y_test = X_test, y_test

data = DATA()
model = CNN(data.input_shape, data.num_classes)
history = model.fit(data.X_train, data.y_train, batch_size=128, epochs=10, validation_split=0.2)
score = model.evaluate(data.X_test, data.y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

plot_acc(history)
plt.show(block=False)
plt.pause(2)
plt.close()

plot_loss(history)
plt.show(block=False)
plt.pause(2)
plt.close()

In [ ]:
# 컬러
import keras.backend
from sklearn import model_selection, metrics
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import os
from keras import backend as K
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from original.keraspp import skeras, sfile

class CNN(Model):
    def __init__(model, nb_classes, in_shape=None):
        model.nb_classes = nb_classes
        model.in_shape = in_shape
        model.build_model()
        super().__init__(model.x, model.y)
        model.cl_part = Model(model.x, model.z_cl)
        model.fl_part = Model(model.x, model.z_fl)
        model.compile()
    
    def build_model(model):
        nb_classes = model.nb_classes
        in_shape = model.in_shape

        x = Input(in_shape)
        h = Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=in_shape)(x)
        h = Conv2D(64, (3,3), activation='relu')(h)
        h = MaxPooling2D(pool_size=(2,2))(h)
        h = Dropout(0.25)(h)
        h = Flatten()(h)
        z_cl = h

        h = Dense(128, activation='relu')(h)
        h = Dropout(0.5)(h)
        z_fl = h

        y = Dense(nb_classes, activation='softmax', name='preds')(h)
        model.z_cl = z_cl
        model.z_fl = z_fl
        model.x, model.y = x, y

    def compile(model):
        Model.compile(model, loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc'])

class DataSet():
    def __init__(self, X, y, nb_classes, scaling=True, test_size=0.2, random_state=0):
        self.X = X
        self.add_channels()
        X = self.X

        X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=random_state)
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')

        if scaling:
            # scaling to have (0, 1) for each feature (each pixel)
            scaler = MinMaxScaler()
            n = X_train.shape[0]
            X_train = scaler.fit_transform(X_train.reshape(n, -1)).reshape(X_train.shape)
            n = X_test.shape[0]
            X_test = scaler.transform(X_test.reshape(n, -1)).reshape(X_test.shape)
            self.scaler = scaler
        
        y_train = np_utils.to_categorical(y_train, nb_classes)
        y_test = np_utils.to_categorical(y_test, nb_classes)
        
        self.X_train, self.X_test = X_train, X_test
        self.y_train, self.y_test = y_train, y_test
    
    def add_channels(self):
        X = self.X

        if len(X.shape) == 3:
            N, img_rows, img_cols = X.shape

            if K.image_data_format() == 'channels_first':
                X = X.reshape(X.shape[0], 1, img_rows, img_cols)
                input_shape = (1, img_rows, img_cols)
            else:
                X = X.reshape(X.shape[0], img_rows, img_cols, 1)
                input_shape = (img_rows, img_cols, 1)
        else:
            input_shape = X.shape[1:]   # channel is already included
        
        self.X = X
        self.input_shape = input_shape

class Machine():
    def __init__(self, X, y, nb_classes=2, fig=True):
        self.nb_classes = nb_classes
        self.set_data(X, y)
        self.set_model()
        self.fig = fig
    
    def set_data(self, X, y):
        nb_classes = self.nb_classes
        self.data = DataSet(X, y, nb_classes)
    
    def set_model(self):
        nb_classes = self.nb_classes
        data = self.data
        self.model = CNN(nb_classes=nb_classes, in_shape=data.input_shape)
    
    def fit(self, nb_epoch=10, batch_size=128, verbose=1):
        data = self.data
        model = self.model
        history = model.fit(data.X_train, data.y_train, batch_size=batch_size, epochs=nb_epoch, verbose=verbose, validation_data=(data.X_test, data.y_test))
        return history

    def run(self, nb_epoch=10, batch_size=128, verbose=1):
        data = self.data
        model = self.model
        fig = self.fig
    
        history = self.fit(nb_epoch=nb_epoch, batch_size=batch_size, verbose=verbose)
        score = model.evaluate(data.X_test, data.y_test, verbose=0)

        print('Confusion matrix')
        y_test_pred = model.predict(data.X_test, verbose=0)
        y_test_pred = np.argmax(y_test_pred, axis=1)
        print(metrics.confusion_matrix(np.argmax(data.y_test, axis=1), y_test_pred))

        print('Test score:', score[0])
        print('Test accuracy:', score[1])

        suffix = sfile.unique_filename('datatime')
        foldname = 'output_' + suffix
        os.makedirs(foldname)
        skeras.save_history_history('history_history.npy', history.history, fold=foldname)
        model.save_weights(os.path.join(foldname, 'dl_model.h5'))
        print('Output results are saved in', foldname)

        if fig:
            plt.figure(figsize=(12,4))
            plt.subplot(1, 2, 1)
            skeras.plot_acc(history)
            plt.subplot(1, 2, 2)
            skeras.plot_loss(history)
            plot_acc(history)
            plt.show(block=False)
            plt.pause(2)
            plt.close()

        self.history = history

from keras.datasets import cifar10
import keras
assert keras.backend.image_data_format() == 'channels_last'
from original.keraspp import aicnn

class Machine(Machine):
    def __init__(self):
        (X, y), (X_test, y_test) = cifar10.load_data()
        super().__init__(X=X, y=y, nb_classes=10)
    
def main():
    m = Machine()
    m.run()

if __name__ == '__main__':
    main()

# 5장 RNN

In [1]:
from __future__ import print_function   # 파이썬 2와 3 간의 호환성 위한 패키지
# 파이썬 2에선 print를 괄호 없이 사용하지만 이 함수를 호출함으로써 함수로써(괄호와 함꼐) 사용하게 함

# 자연어
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.datasets import imdb
from keras import layers, models

class Data:
    def __init__(self, max_features=20000, maxlen=80):
        (x_train, self.y_train), (x_test, self.y_test) = imdb.load_data(num_words=max_features)

        # x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
        # x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
        # pad_sequences의 경로가 바뀜
        self.x_train = pad_sequences(x_train, maxlen=maxlen)
        self.x_test = pad_sequences(x_test, maxlen=maxlen)

class RNN_LSTM(models.Model):
    def __init__(self, max_features, maxlen):
        # model = models.Sequential()
        # model.add(layers.Embedding(max_features, 128))
        # model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
        # model.add(layers.Dense(1, activation='sigmoid'))
        # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

        x = layers.Input((maxlen, ))
        h = layers.Embedding(max_features, 128)(x)
        h = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(h)
        y = layers.Dense(1, activation='sigmoid')(h)
        super().__init__(x, y)
        self.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

class Machine:
    def __init__(self, max_features=20000, maxlen=80):
        self.data = Data(max_features, maxlen)
        self.model = RNN_LSTM(max_features, maxlen)
    
    def run(self, epochs=3, batch_size=32):
        data = self.data
        model = self.model

        print('Training stage')
        print('====================')
        model.fit(data.x_train, data.y_train, batch_size=batch_size, epochs=epochs, validation_data=(data.x_test, data.y_test))
        score, acc = model.evaluate(data.x_test, data.y_test, batch_size=batch_size)
        print('Test performance: accuracy{0}, loss={1}'.format(acc, score))

def main():
    m = Machine()
    m.run(batch_size=512)

if __name__ == '__main__':
    main()

Training stage
Epoch 1/3
49/49 [==============================] - 19s 341ms/step - loss: 0.6020 - acc: 0.6710 - val_loss: 0.4355 - val_acc: 0.8116
Epoch 2/3
49/49 [==============================] - 18s 370ms/step - loss: 0.3275 - acc: 0.8616 - val_loss: 0.3542 - val_acc: 0.8429
Epoch 3/3
49/49 [==============================] - 2s 37ms/step - loss: 0.3765 - acc: 0.8352
Test performance: accuracy0.8352400064468384, loss=0.3765231966972351


In [ ]:
# 시계열
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras import models, layers

from original.keraspp import skeras

def main():
    machine = Machine()
    machine.run(epochs=400)

class Machine():
    def __init__(self):
        self.data = Dataset()
        shape = self.data.X.shape[1:]
        self.model = rnn_model(shape)
    
    def run(self, epochs=400):
        d = self.data
        X_train, X_test = d.X_train, d.X_test
        y_train, y_test = d.y_train, d.y_test
        X, y = d.X, d.y
        
        m = self.model
        h = m.fit(X_train, y_train, epochs=epochs, validation_data=[X_test, y_test], verbose=0)
        skeras.plot_loss(h)
        plt.title('History of training')
        plt.show(block=False)
        plt.pause(2)
        plt.close()

        yp = m.predict(X_test)
        print('Loss:', m.evaluate(X_test, y_test))
        plt.plot(yp, label='Original')
        plt.plot(y_test, label='Prediction')
        plt.legend(loc=0)
        plt.title('Validation Results')
        plt.show(block=False)
        plt.pause(2)
        plt.close()

        yp = m.predict(X_test).reshape(-1)
        print('Loss:', m.evaluate(X_test, y_test))
        print(yp.shape, y_test.shape)
        df = pd.DataFrame()
        df['Sample'] = list(range(len(y_test))) * 2
        df['Normalized #Passengers'] = np.concatenate([y_test, yp], axis=0)
        df['Type'] = ['Original'] * len(y_test) + ['Prediction'] * len(yp)

        plt.figure(figsize=(7, 5))
        sns.barplot(x='Sample', y='Normalized #Passengers', hue='Type', data=df)
        plt.ylabel('Normalized #Passengers')
        plt.show(block=False)
        plt.pause(2)
        plt.close()

        yp = m.predict(X)
        plt.plot(yp, label='Original')
        plt.plot(y, label='Prediction')
        plt.legend(loc=0)
        plt.title('All Results')
        plt.show(block=False)
        plt.pause(2)
        plt.close()

def rnn_model(shape):
    m_x = layers.Input(shape=shape)
    m_h = layers.LSTM(10)(m_x)
    m_y = layers.Dense(1)(m_h)
    m = models.Model(m_x, m_y)
    m.compile('adam', 'mean_squared_error')
    m.summary()

    return m

class Dataset:
    def __init__(self, fname='original/international-airline-passengers.csv', D=12):
        data_dn = load_data(fname=fname)
        X, y = get_Xy(data_dn, D=D)
        X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)
        self.X, self.y = X, y
        self.X_train, self.X_test = X_train, X_test
        self.y_train, self.y_test = y_train, y_test
    
def load_data(fname='original/international-airline-passengers.csv'):
    dataset = pd.read_csv(fname, usecols=[1], engine='python', skipfooter=3)
    data = dataset.values.reshape(-1)

    plt.plot(data)
    plt.xlabel('Time')
    plt.ylabel('#Passengers')
    plt.title('Original Data')
    plt.show(block=False)
    plt.pause(2)
    plt.close()

    data_dn = (data - np.mean(data)) / np.std(data) / 5
    plt.plot(data_dn)
    plt.xlabel('Time')
    plt.ylabel('Normalized #Passengers')
    plt.title('Normalized data by $E[]$ and $5\sigma$')
    plt.show(block=False)
    plt.pause(2)
    plt.close()
    
    return data_dn

def get_Xy(data, D=12):
    # make X and y
    X_l = []
    y_l = []
    N = len(data)
    assert N>D, 'N should be larger than D, where N is len(data)'
    for ii in range(N-D-1):
        X_l.append(data[ii:ii+D])
        y_l.append(data[ii+D])
    X = np.array(X_l)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    y = np.array(y_l)
    print(X.shape, y.shape)

    return X, y


if __name__ == '__main__':
    main()